Статья написана группой известных исследователей в области компьютерного зрения и описывает два необычных и контр-интуитивных свойства, обнаруженных у нейронных сетей.

Предметом исследований являются:
1. Обученная на ImageNet сверточная сеть [AlexNet]($ImageNet Classification with Deep Convolutional Neural Networks$)
2. Обученная на MNIST глубокая полносвязная сеть (MLP)
3. Обученный на кадрах с YouTube глубокий автоэнкодер [QuocNet]($Building high-level features using large scale unsupervised learning$)

## Интерпретируемость нейронов

Первое свойство касается того, имеют ли отдельные юниты (нейроны в MLP, фильтры в CNN) какой-то явный смысл. Было бы соблазнительно предположить, что один нейрон может отвечать за лапу, другой за глаз, третий за шерсть, и все вместе они определяют на изображении кошку. Это бы соответствовало человеческому мышлению: объект собирается из составляющих его частей.

В более ранних работах уже делались попытки интерпретации значений фильтров сверточной сети. Для этого в тестовом датасете отыскивались изображения, дающие максимальную активацию данного юнита. В полученном наборе изображений исследователи пытались найти что-то общее. И действительно, обнаруживались некоторые закономерности. Один из фильтров активировался в основном на тех изображениях, где изображены круги и спирали, другой на тех, где изображены сетки или соты, третий на тех, где изображены человеческие силуэты, и так далее (например см. [эту статью]($Visualizing and Understanding Convolutional Networks$), стр. 4).

Авторы текущей статьи предложили искать изображения, на которых достигают максимума не отдельные фильтры, а их комбинации. Для этого мы извлекаем из изображения или его участка вектор признаков $\mathbf{\phi(x)}$ (то есть вектор активаций фильтров), а затем считаем скалярное произведение этого вектора с неким другим заранее выбранным вектором $\mathbf{v}$, получая число. Мы ищем в тестовом датасете те изображения, где это число достигает максимума.

Например, если вектор $\mathbf{v}$ содержит только одну единицу, а все остальные элементы нулевые, то задача сводится к максимизации отклика одного фильтра. Если в векторе $\mathbf{v}$ две единицы, тогда задача означает максимизацию суммы откликов двух фильтров. Но в общем случае вектор $\mathbf{v}$ может состоять из случайных чисел. Если представить геометрически, то это означает поиск в тестовом датасете изображений, уходящих максимально далеко в пространстве признаков по некоему направлению, задаваемому вектором $\mathbf{v}$.

Как оказалось, при любом векторе $\mathbf{v}$ мы получаем похожую картину: найденные изображения часто имеют что-то общее. Иными словами, данный эксперимент не обнаруживает никакой разницы между отдельно взятыми фильтрами (нейронами) и их произвольными линейными комбинациями.

Это наводит авторов на мысль, что информация, извлекаемая нейронной сетью, заложена во всем пространстве признаков, то есть всех фильтрах сразу, а не в каждом по отдельности. При этом каждый отдельно взятый фильтр не интерпретируем.

> *“This puts into question the conjecture that neural
networks disentangle variation factors across coordinates. Generally, it seems that it is the entire
space of activations, rather than the individual units, that contains the bulk of the semantic information.”*

**Мои мысли по этому поводу**

Для иллюстрации давайте абстрагируемся от нейронных сетей и представим просто набор признаков, каждый из которых имеет некий явный смысл. Например оценки за учебный год по 10 предметам. Такие признаки можно назвать "disentangled features". Они формируют 10-мерное пространство признаков. Теперь осуществим какую-нибудь сложную деформацию этого пространства, как если бы мы взяли кусок пластилина и смяли его. Также мы можем "разрезать" это пространство на части и склеить их в другом порядке. Преобразование, которое мы выполнили, биективно: то есть по координате в новом пространстве можно однозначно восстановить координату в старом пространстве. Это означает, что информация не утеряна. Но в новом пространсте признаков каждая координата перестала нести явный смысл. Признаки стали "спутанными" (entangled). В текущей статья как раз и делается предположение, что признаки, извлекаемые нейронной сетью, "спутаны".

От себя добавлю, что есть и еще одно подтверждение этой точки зрения. Если мы визуализируем веса сверточных слоев (например, с помощью `plt.imshow`), то увидим, что близких к нулю значений очень мало. Это говорит о том, что уровень активации каждого сверточного фильтра влияет практически на все фильтры следующего слоя. Точно так же, в полносвязной сети каждый нейрон предыдущего слоя влияет практически на каждый нейрон следующего слоя. А если предположить, что нейроны означают по смыслу какие-то объекты или их части, то их влияние друг на друга по-видимому должно быть *разреженным (sparse connectivity)*, то есть лишь несколько фильтров предыдущего уровня должны вносить доминирующий вклад в активацию фильтра следующего уровня (кот = лапы + уши + шерсть + ...). На деле же мы видим *плотную связность (dense connectivity)*, которая может говорить о том, что признаки неинтерпретируемы. В целом о разреженности и ее связи с сознательным интеллектом, в противоположность бессознательному интеллекту, говорится в статье [The Consciousness Prior](https://arxiv.org/pdf/1709.08568.pdf) (Yoshua Bengio, 2017), на которую я делал [обзор]($The Consciousness Prior$).

## Adversarial examples

Авторы начинают с того, что упоминают [книгу]($Learning Deep Architectures for AI$) Йошуа Бенжио "Learning Deep Architectures for AI", в которой предполагается, что глубина нейронной сети позволяет достичь нелокальной генерализации (non-local generalization). Это понятие означает корректную обработку примеров, которые существенно отличались от всех встречаемых при обучении, например, тот же объект с другого ракурса. При этом более легкой задачей считается локальная генерализация, то есть корректная обработка примеров, которые незначительно отличаются от одного из обучающих примеров.

Тем не менее, в работе показано, что даже локальная генерализация достигается не полностью. Обучающее изображение можно лишь незначительно изменить так, что сеть перестанет его корректно классифицировать.

Например, пусть наша нейронная сеть решает задачу классификации, и среди классов присутствуют "кошка" и "автомобиль". Мы имеем фотографию кошки и хотим, чтобы нейронная сеть распознавала на этой фотографии автомобиль, лишь незначительно изменив эту фотографию, то есть наложив на нее шум, не различимый человеческим глазом. Задача решается градиентным спуском. Перед началом градиентного спуска инициализируется массив с шумом $\textbf{r}$ (массив чисел, размером эквивалентный изображению и заполненный нулями). Шаг градиентного спуска состоит в том, что сумма изображения и шума пропускается через нейронную сеть, после чего рассчитывается две функции потерь:

1. Кроссэнтропия между предсказаниями сети и верным ответом "автомобиль"
2. L2-норма шума $\textbf{r}$, то есть корень из суммы квадратов всех его элементов

Затем считается взвешенная сумма двух этих функций потерь (с некими заранее выбранными весами), расчитывается градиент суммарной функции потерь по шуму $\textbf{r}$, после чего $\textbf{r}$ обновляется. При этом веса нейронной сети не меняются. Таким образом, мы решаем задачу оптимизации, достигая одновременно двух целей: чтобы шум был незначительным, и чтобы наложение этого шума на изображение кошки заставляло сеть выдавать ответ "автомобиль".

В итоге всех трех сетей (MLP, AlexNet, QuocNet) авторам удалось заставить нейронную сеть ошибаться наложением на изображения незначительного, не видимого глазу шума. Еще более неожиданным оказалось то, что в точности тот же шум заставлял ошибаться на том же изображении и другие сети, обученные с другими гиперпараметрами и на других подвыборках обучающих данных.

Авторам удалось также обратить найденный недостаток на пользу, добавив зашумленные неправильно классифицируемые изображения в массив обучающих данных.

В конце статьи делается попытка подойти к проблеме adversarial examples с математической точки зрения. Для этого расчитывается коэффициент стабильности сети, который ограничен сверху произведением констант Липшица всех слоев сети. Делается предположение, что "навешивая" при обучении на каждый слой модели дополнительный loss, равный константе Липшица этого слоя, можно будет избавиться от данной проблемы.

В заключение авторы оставляют для будущих исследований вопрос о том, насколько существенна найденная проблема. С одной стороны, adversarial examples могут быть исчезающе редкими в реальных данных, но с другой стороны такие примеры, будь они найдены, могли бы использоваться для "атак" на нейронные сети, вынуждая их давать неверные ответы.